This project explores a realistic approach to modelling market information from different sources.

The provided dataset contains information on: 

- Equity prices and volumes

- Option Greeks and implied volatility

- Realized volatility

- Sentiment data

- Market indices

Our goals in this project are to explore feature engineering, cross model correlations, and predictive modeling

Along the way we will also learn about key financial components such as volatility and sentiment and relavent considerations 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
df = pd.read_csv("data/data.csv", parse_dates=["date","expiration_date"])
cols = df.columns
cols

Index(['date', 'volume_of_trades', 'strike', 'stock_close', 'stock_high',
       'stock_low', 'stock_open', 'stock_traded_volume', 'divCash',
       'expiration_date', 'options_close_price', 'options_volume', 'count',
       'bid', 'bid_size', 'ask', 'ask_size', 'open_interest', 'delta', 'theta',
       'vega', 'rho', 'epsilon', 'lambda', 'gamma', 'd1', 'd2', 'implied_vol',
       'iv_error', 'expiration', 'realized_vol', 'realized_vol_diff_target',
       '7_day_realized_vol_target', 'realized_vol_diff_bin_target',
       '7_day_implied_vol_target', 'implied_vol_diff_target',
       'implied_vol_diff_bin_target', 'options_close_price_7_days',
       'options_7_day_diff', 'options_7_day_frac_diff',
       'options_7_day_diff_bin', '7_day_implied_7_day_forecasted_vol_diff',
       '7_day_implied_7_day_forecasted_vol_diff_bin',
       'current_implied_7_day_forecasted_vol_diff',
       'current_implied_7_day_forecasted_vol_diff_bin',
       'reported_estimate_eps_percent_diff', 'vix-open

We have a lot of features to consider here.

One useful approach to feature engineering is to group our features based on their domains.

We see that in our data we can naturally group our data by:

- stock

- option

- greeks

- volatility

- sentiment

- earnings

- vix

In [33]:
feature_groups = {
    "stock": ['stock_open','stock_high','stock_low','stock_close','stock_traded_volume'],
    "option": ['options_close_price','options_volume','strike','open_interest'],
    "greeks": ['delta','gamma','theta','vega','rho','epsilon','lambda','d1','d2'],
    "volatility": ['implied_vol','realized_vol','realized_vol_diff_target','7_day_realized_vol_target','7_day_implied_vol_target','implied_vol_diff_target'],
    "vix": ['vix-open','vix-high','vix-low','vix-close'],
    "sentiment": ['article_sentiment','pos_total_count','neu_total_count','neg_total_count','total_count'],
    "earnings": ['reported_estimate_eps_percent_diff','pos_em_count','neg_em_count','em_total_count'],
}
for group, cols in feature_groups.items():
    print(f"\n{group} features: {len(cols)}")
    display(df[cols].describe().T.head(10))


stock features: 5


,count,mean,std,min,25%,50%,75%,max
stock_open,6252.0,1.011236e+02,9.529625e+00,8.509091e+01,9.439710e+01,9.948861e+01,1.061942e+02,1.251853e+02
stock_high,6252.0,1.025950e+02,9.602983e+00,8.620685e+01,9.568243e+01,1.008238e+02,1.074597e+02,1.259724e+02
stock_low,6252.0,9.993549e+01,9.578786e+00,8.303836e+01,9.326123e+01,9.844241e+01,1.047096e+02,1.222958e+02
stock_close,6252.0,1.012774e+02,9.662109e+00,8.312804e+01,9.433732e+01,9.977756e+01,1.069515e+02,1.245974e+02
stock_traded_volume,6252.0,3.380256e+07,1.438074e+07,9.701441e+06,2.499427e+07,3.041104e+07,3.700040e+07,1.194550e+08



option features: 4


,count,mean,std,min,25%,50%,75%,max
options_close_price,6252.0,3.263034,1.001214,0.68,2.53,3.2,3.89,7.24
options_volume,6252.0,1375.029750,2577.556678,11.00,173.00,532.0,1449.00,37504.00
strike,6252.0,101.613884,9.660514,84.00,95.00,100.0,106.00,126.00
open_interest,6234.0,2621.408566,5759.041140,0.00,314.25,787.0,2072.00,42368.00



greeks features: 9


...

,count,mean,std,min,25%,50%,75%,max
vix-open,6252.0,22.678141,4.227956,16.13,19.39,21.80,25.34,34.50
vix-high,6252.0,23.579760,4.396253,16.62,20.08,22.60,26.35,34.88
vix-low,6252.0,21.714399,4.030704,15.53,18.80,20.89,23.85,33.11
vix-close,6252.0,22.351256,4.190005,15.78,19.10,21.44,25.00,33.63



sentiment features: 5


,count,mean,std,min,25%,50%,75%,max
article_sentiment,6023.0,-0.045160,0.688873,-5.0,0.0,0.0,0.0,6.0
pos_total_count,6023.0,488.933588,406.929067,0.0,131.0,477.0,628.0,2034.0
neu_total_count,6023.0,148.426864,146.252667,0.0,48.0,133.0,185.0,1122.0
neg_total_count,6023.0,223.132658,222.471110,0.0,64.0,206.0,278.0,1602.0
total_count,6023.0,860.493110,733.347301,0.0,227.0,832.0,1099.0,4428.0



earnings features: 4


,count,mean,std,min,25%,50%,75%,max
reported_estimate_eps_percent_diff,6252.0,-0.066286,1.348195,-15.2,0.0,0.0,0.0,10.377358
pos_em_count,6023.0,153.973933,148.913652,0.0,59.0,129.0,193.5,1099.000000
neg_em_count,6023.0,52.748962,59.177910,0.0,20.0,44.0,64.0,439.000000
em_total_count,6023.0,230.251038,229.576494,0.0,77.0,204.0,275.0,1643.000000


Similar to last week, lets get a visual interpretation of our data by looking at a correlation heatmap of our different variables.

In [34]:
corr_cols = ['stock_close','options_close_price','implied_vol','realized_vol','article_sentiment','vix-close']
corr_matrix = df[corr_cols].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Cross-Domain Correlation Matrix")
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'